In [1]:
import fiftyone as fo
import fiftyone.zoo as foz
import os
COCO_CLASSES = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
        'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
        'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
        'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
        'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
        'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
        'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
        'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
        'hair drier', 'toothbrush']

In [2]:
safety_trigger = False

In [3]:
PROJECT_NAME = 'person'
ROOT_DIR = '/home/mosminin/datasets/COCO2017'



coco_dataset_dir = os.path.join(ROOT_DIR, 'coco2017_all_classes')
dataset_name = f'coco_{PROJECT_NAME}'

# Настройки экспорта
TARGET_CLASSES = ['person'] # Список нужных классов
yolov4_export_dir = os.path.join(ROOT_DIR, f'coco2017_yolov4_{PROJECT_NAME}') # Целевая папка
label_field = "ground_truth"

# ПАРСИНГ ДАТАСЕТА COCO-2017 ДЛЯ YOLOV4

## СКАЧИВАЕМ ПОЛНЫЙ ДАТАСЕТ COCO-2017

In [18]:
if not safety_trigger:
    # Выполнение кода грохнет текущий датасет и начнёт долгое скачивание заново!
    dataset = foz.load_zoo_dataset(
        "coco-2017",
        splits=["train", "validation", "test"],
        dataset_dir=coco_dataset_dir,
        dataset_name=dataset_name,
        dataset_type=fo.types.COCODetectionDataset,
        classes=COCO_CLASSES,
        drop_existing_dataset=True,
        # only_matching=False,
        shuffle=True,
        max_samples=1000
    )
    safety_trigger = True
else:
    print('Check safety trigger!')

Found annotations at '/home/mosminin/datasets/COCO2017/coco2017_all_classes/raw/instances_train2017.json'
Sufficient images already downloaded
Existing download of split 'train' is sufficient
Found annotations at '/home/mosminin/datasets/COCO2017/coco2017_all_classes/raw/instances_val2017.json'
Sufficient images already downloaded
Existing download of split 'validation' is sufficient
Test split is unlabeled; ignoring classes requirement
Found test info at '/home/mosminin/datasets/COCO2017/coco2017_all_classes/raw/image_info_test2017.json'
Sufficient images already downloaded
Existing download of split 'test' is sufficient
Ignoring unsupported parameter 'dataset_type' for importer type <class 'fiftyone.utils.coco.COCODetectionDatasetImporter'>
Deleting existing dataset 'coco_person'
Loading 'coco-2017' split 'train'
 100% |███████████████| 1000/1000 [2.8s elapsed, 0s remaining, 365.9 samples/s]      
Loading 'coco-2017' split 'validation'
 100% |███████████████| 1000/1000 [2.6s elapsed,

## ПООЧЕРЁДНО ЗАГРУЖАЕМ КАЖДЫЙ СПЛИТ И ЭКСПОРТИРУЕМ В СООТВЕТСТВУЮЩУЮ ПАПКУ

In [5]:
dataset = fo.load_dataset(dataset_name)

In [14]:
import shutil
from tqdm.notebook import tqdm
from pathlib import Path
from time import sleep

splits = ['train', 'validation', 'test'] # Splits for export

for split in splits:
    split_view = dataset.match_tags(split)
    if split == 'validation':
        split = 'valid'  # For Yolo compatibility
    print(f'{split=}')
    export_dir = Path(os.path.join(yolov4_export_dir, split))
    split_view.export(
        export_dir=str(export_dir),
        dataset_type=fo.types.YOLOv4Dataset,
        label_field=label_field,
        classes=TARGET_CLASSES,
    )
    data_path = Path(os.path.join(yolov4_export_dir, split,'data'))  # Abs path

    annotations_file = export_dir / f'{split}.txt'

    print('Path: ', str(data_path))

    for file_path in data_path.glob('*.jpg'):
        print(str(f'{file_path=}'))

    # Читаем все .jpg файлы в /data
    file_list = []
    for file_path in data_path.glob('*.jpg'):
        print(f'{file_path=}')
        file_list.append(str(file_path.resolve()))

    with open(annotations_file, 'w') as f:
        for file_path in file_list:
            print(f'{file_path=}')
            f.write(file_path+'\n')



    # # Переносим файлы из папки {split}/data на уровень выше
    # data_path = os.path.join(yolov4_export_dir, 'data')  # Abs path
    # print(f'{data_path=}')
    # with open(os.path.join(export_dir, f'{split}.txt'), 'w') as annotations:
    #     for f in tqdm(os.listdir(data_path)):
    #         src_file = os.path.join(data_path, f)
    #         dst_file = os.path.join(export_dir, f)
    #         if os.path.exists(dst_file):
    #             # File already exists, decide what to do
    #             print(f"Skipping {f} as it already exists in {export_dir}")
    #         else:
    #             # Move the file to the export_dir
    #             shutil.move(src_file, export_dir)
    #             if f.endswith('.jpg'):
    #                 filepath = os.path.join()
    # shutil.rmtree(data_path)




split='train'
Directory '/home/mosminin/datasets/COCO2017/coco2017_yolov4_person/train' already exists; export will be merged with existing files


/home/mosminin/.virtualenvs/yolo4scaled/lib/python3.9/site-packages/fiftyone/utils/yolo.py:991: UserWarning: Ignoring detection with label 'teddy bear' not in provided classes
  warnings.warn(msg)
/home/mosminin/.virtualenvs/yolo4scaled/lib/python3.9/site-packages/fiftyone/utils/yolo.py:991: UserWarning: Ignoring detection with label 'banana' not in provided classes
  warnings.warn(msg)
/home/mosminin/.virtualenvs/yolo4scaled/lib/python3.9/site-packages/fiftyone/utils/yolo.py:991: UserWarning: Ignoring detection with label 'clock' not in provided classes
  warnings.warn(msg)
/home/mosminin/.virtualenvs/yolo4scaled/lib/python3.9/site-packages/fiftyone/utils/yolo.py:991: UserWarning: Ignoring detection with label 'refrigerator' not in provided classes
  warnings.warn(msg)
/home/mosminin/.virtualenvs/yolo4scaled/lib/python3.9/site-packages/fiftyone/utils/yolo.py:991: UserWarning: Ignoring detection with label 'chair' not in provided classes
  warnings.warn(msg)
/home/mosminin/.virtualenvs

  29% |████|----------|  289/1000 [447.8ms elapsed, 1.1s remaining, 645.3 samples/s] 

/home/mosminin/.virtualenvs/yolo4scaled/lib/python3.9/site-packages/fiftyone/utils/yolo.py:991: UserWarning: Ignoring detection with label 'frisbee' not in provided classes
  warnings.warn(msg)
/home/mosminin/.virtualenvs/yolo4scaled/lib/python3.9/site-packages/fiftyone/utils/yolo.py:991: UserWarning: Ignoring detection with label 'bird' not in provided classes
  warnings.warn(msg)
/home/mosminin/.virtualenvs/yolo4scaled/lib/python3.9/site-packages/fiftyone/utils/yolo.py:991: UserWarning: Ignoring detection with label 'motorcycle' not in provided classes
  warnings.warn(msg)
/home/mosminin/.virtualenvs/yolo4scaled/lib/python3.9/site-packages/fiftyone/utils/yolo.py:991: UserWarning: Ignoring detection with label 'keyboard' not in provided classes
  warnings.warn(msg)
/home/mosminin/.virtualenvs/yolo4scaled/lib/python3.9/site-packages/fiftyone/utils/yolo.py:991: UserWarning: Ignoring detection with label 'snowboard' not in provided classes
  warnings.warn(msg)


  43% |██████---------|  434/1000 [649.8ms elapsed, 847.5ms remaining, 667.8 samples/s] 

/home/mosminin/.virtualenvs/yolo4scaled/lib/python3.9/site-packages/fiftyone/utils/yolo.py:991: UserWarning: Ignoring detection with label 'bear' not in provided classes
  warnings.warn(msg)
/home/mosminin/.virtualenvs/yolo4scaled/lib/python3.9/site-packages/fiftyone/utils/yolo.py:991: UserWarning: Ignoring detection with label 'hair drier' not in provided classes
  warnings.warn(msg)
/home/mosminin/.virtualenvs/yolo4scaled/lib/python3.9/site-packages/fiftyone/utils/yolo.py:991: UserWarning: Ignoring detection with label 'toaster' not in provided classes
  warnings.warn(msg)
/home/mosminin/.virtualenvs/yolo4scaled/lib/python3.9/site-packages/fiftyone/utils/yolo.py:991: UserWarning: Ignoring detection with label 'parking meter' not in provided classes
  warnings.warn(msg)


 100% |███████████████| 1000/1000 [1.5s elapsed, 0s remaining, 712.3 samples/s]         
Reading
Path:  /home/mosminin/datasets/COCO2017/coco2017_yolov4_person/train/data
file_path=PosixPath('/home/mosminin/datasets/COCO2017/coco2017_yolov4_person/train/data/000000007890.jpg')
file_path=PosixPath('/home/mosminin/datasets/COCO2017/coco2017_yolov4_person/train/data/000000027842.jpg')
file_path=PosixPath('/home/mosminin/datasets/COCO2017/coco2017_yolov4_person/train/data/000000001966.jpg')
file_path=PosixPath('/home/mosminin/datasets/COCO2017/coco2017_yolov4_person/train/data/000000017179.jpg')
file_path=PosixPath('/home/mosminin/datasets/COCO2017/coco2017_yolov4_person/train/data/000000021083.jpg')
file_path=PosixPath('/home/mosminin/datasets/COCO2017/coco2017_yolov4_person/train/data/000000025550.jpg')
file_path=PosixPath('/home/mosminin/datasets/COCO2017/coco2017_yolov4_person/train/data/000000020193.jpg')
file_path=PosixPath('/home/mosminin/datasets/COCO2017/coco2017_yolov4_person/tra

## СКРИПТ ДЛЯ РАСПРЕДЕЛЕНИЯ ДАТАСЕТА

- dataset   - valid - .jpg / .txt
            - train - .jpg / .txt
            - test  - .jpg

    - Для каждой папки перебрать .jpg и составить список train/valid/test.txt
    - Файлы со списками сохранить в dataset
    - В dataset папку backup
    - В dataset создать файл obj.data:
        classes =   {кол-во_классов}
        train =     {абсолютный пусть до train.txt}
        valid =     ...
        names =     {абсолютный пусть до obj.names}
        backup =    {абсолютный пусть до папки backup}

# БЭКАПИЛКА

In [ ]:
import os
import time
import tarfile

def make_archive(dir_path):
    now = time.strftime('%Y_%m_%d_%H%M%S')
    archive_name = f'{dir_path}_{now}.tar'
    i = 0
    while os.path.exists(archive_name):
        i += 1
        archive_name = f'{dir_path}_{now}_{i}.tar'
    with tarfile.open(archive_name, 'w') as archive:
        archive.add(dir_path, arcname=os.path.basename(dir_path), mode='a')